In [6]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from IPython.core.interactiveshell import InteractiveShell
from sklearn.metrics import accuracy_score
from hmmlearn import hmm

from fetch import fetch_ORL_people
InteractiveShell.ast_node_interactivity = "all"
np.set_printoptions(precision=3)

In [7]:
faces = fetch_ORL_people('../Data')
faces.keys()

dict_keys(['data', 'images', 'target', 'target_names'])

In [11]:
if random_split:
    idx_train, idx_test = train_test_split(np.arange(faces.images.shape[0]), test_size=0.5, stratify=faces.target, random_state=None)
else:
    idx_train = np.tile([0,4,5,7,9], n_person)
    idx_test = np.tile([1,2,3,6,8], n_person)
    for i in range(n_person):
        idx_train[i*5:i*5+5] += i*10
        idx_test[i*5:i*5+5] += i*10

train_faces = faces.images[idx_train]
y_train = faces.target[idx_train]
test_faces = faces.images[idx_test]
y_test = faces.target[idx_test]

In [10]:
coeff = np.array([18., 10., 7.])
eps = 1e-6
n_state = 7
n_obs = 1260
n_person = 40
random_split = False

In [12]:
def split_block_cell_for_train(faces, n_blocks=52):
    n_faces = faces.shape[0]
    max_coeffs = np.array([-np.inf, -np.inf, -np.inf])
    min_coeffs = np.array([np.inf, 0, 0])
    block_cell = np.zeros(shape=(n_faces, n_blocks, 3))

    for faces_index in range(n_faces):
        img = faces[faces_index]
        for block_index in range(n_blocks):
            block = img[block_index:block_index + 5]
            U, S, V = np.linalg.svd(block)
            block_coeffs = np.array([U[0,0], S[0], S[1]])
            min_coeffs = np.minimum(block_coeffs, min_coeffs)
            max_coeffs = np.maximum(block_coeffs, max_coeffs)
            block_cell[faces_index, block_index] = block_coeffs
            
    return min_coeffs, max_coeffs, block_cell

In [13]:
def split_block_cell_for_test(faces, min_coeffs, max_coeffs, n_blocks=52):
    n_faces = faces.shape[0]
    block_cell = np.zeros(shape=(n_faces, n_blocks, 3))
    
    for faces_index in range(n_faces):
        img = faces[faces_index]
        for block_index in range(n_blocks):
            block = img[block_index:block_index + 5]
            U, S, V = np.linalg.svd(block)
            block_coeffs = np.array([U[0,0], S[0], S[1]])
            block_coeffs = np.minimum(block_coeffs, max_coeffs)
            block_coeffs = np.maximum(block_coeffs, min_coeffs)
            block_cell[faces_index, block_index] = block_coeffs
            
    return block_cell

In [14]:
def block_to_seq(block_cell, min_coeffs, delta):
    n_faces, n_blocks, _ = block_cell.shape
    seq_matrix = np.zeros(shape=(n_faces, n_blocks))
    for faces_index in range(n_faces):
        for block_index in range(n_blocks):
            block_coeffs = block_cell[faces_index, block_index]
            Q_t = np.floor((block_coeffs - min_coeffs) / delta)
            label = Q_t[0]*10*7 + Q_t[1]*7 + Q_t[2]
            seq_matrix[faces_index, block_index] = label
    return seq_matrix

In [15]:
min_coeffs, max_coeffs, train_block = split_block_cell_for_train(train_faces)
delta = (max_coeffs - min_coeffs) / (coeff - eps)
seq_train = block_to_seq(train_block, min_coeffs, delta)
seq_train = seq_train.astype(np.int32)


In [16]:
test_block = split_block_cell_for_test(test_faces, min_coeffs, max_coeffs)
seq_test = block_to_seq(test_block, min_coeffs, delta)
seq_test = seq_test.astype(np.int32)

In [17]:
startprob = np.zeros(n_state)
startprob[0] = 1.

transmat = np.zeros([n_state, n_state]) + eps
transmat[-1, -1] = 1.
for i in range(n_state-1):
    transmat[i, i] = 0.6
    transmat[i, i+1] = 0.4

transmat /= transmat.sum(1, keepdims=True)
emissionprob = np.ones([n_state, n_obs]) / n_obs


print(f'StartProb Shape {startprob.shape}:\n {startprob}')
print(f'Transmat Shape {transmat.shape}:\n {transmat}')
print(f'Emissionprob Shape {emissionprob.shape}:\n {emissionprob}')

StartProb Shape (7,):
 [1. 0. 0. 0. 0. 0. 0.]
Transmat Shape (7, 7):
 [[6.e-01 4.e-01 1.e-06 1.e-06 1.e-06 1.e-06 1.e-06]
 [1.e-06 6.e-01 4.e-01 1.e-06 1.e-06 1.e-06 1.e-06]
 [1.e-06 1.e-06 6.e-01 4.e-01 1.e-06 1.e-06 1.e-06]
 [1.e-06 1.e-06 1.e-06 6.e-01 4.e-01 1.e-06 1.e-06]
 [1.e-06 1.e-06 1.e-06 1.e-06 6.e-01 4.e-01 1.e-06]
 [1.e-06 1.e-06 1.e-06 1.e-06 1.e-06 6.e-01 4.e-01]
 [1.e-06 1.e-06 1.e-06 1.e-06 1.e-06 1.e-06 1.e+00]]
Emissionprob Shape (7, 1260):
 [[0.001 0.001 0.001 ... 0.001 0.001 0.001]
 [0.001 0.001 0.001 ... 0.001 0.001 0.001]
 [0.001 0.001 0.001 ... 0.001 0.001 0.001]
 ...
 [0.001 0.001 0.001 ... 0.001 0.001 0.001]
 [0.001 0.001 0.001 ... 0.001 0.001 0.001]
 [0.001 0.001 0.001 ... 0.001 0.001 0.001]]


In [18]:
models = [hmm.MultinomialHMM(n_components=n_state, tol=1e-2, n_iter=5, init_params='') for _ in range(n_person)]
for model in models:
    model.startprob_ = startprob
    model.transmat_ = transmat
    model.emissionprob_ = emissionprob
    model.n_features = n_obs

In [24]:
%%time
for person_idx, model in enumerate(models):
    seq_idx = np.where(y_train==person_idx)[0]
    obs = seq_train[seq_idx]
    model.fit(obs.T)

ValueError: Found array with 0 feature(s) (shape=(52, 0)) while a minimum of 1 is required.

In [ ]:
%%time
logprob = np.zeros((y_test.size, n_person))
for seq_idx, obs in enumerate(seq_test):
    for person_idx, model in enumerate(models):
        logprob[seq_idx, person_idx] = model.score(np.atleast_2d(obs))